In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pickle
import copy
import nengo
import nengo_rt_backend
%matplotlib inline

In [2]:
hardware = True

model = nengo.Model('Motion', seed=1)

In [3]:
inputs = pickle.load(open("/home/mtrberzi/dot-images-coh1-s02-inputs.p", "rb" ))

dt = .001

class InputWrapper: 
    def __init__(self, inputs, index): 
        self.inputs = inputs
        self.index = index
    
    def __call__(self, t): 
        step_num = np.round(t/dt)
        # clamp to maximum size of array
        if step_num >= self.inputs.shape[2]:
            step_num = self.inputs.shape[2] - 1 
        return self.inputs[self.index,:,step_num] / 20
        
# n = inputs.shape[0]
n = 1

input_nodes = []
input_probes = []
for i in range(n): 
    wrapper = InputWrapper(inputs, i)
    node = nengo.Node(output=wrapper, size_in=0, size_out=inputs.shape[1], label="Gabor"+str(i))
    input_nodes.append(node)
    # input_probes.append(nengo.Probe(node, 'output'))

In [4]:
n_dirs = 2 #Number of V1 oscillator directions
n_speeds = 3 #Number of V1 oscillator speeds
tcs = np.array([65,100,160])*0.001 #Time constants corresponding to the 20... ms half periods in the thesis for speeds
# ...except where they don't because using 40 instead of 65 gives an unrepresentable value

MT_tau = 0.02

def V1_subpopulation():
    """
    This function builds subnets of n_dir x n_speeds populations that need to be connected to the
    input of aligned LGN elements in a direction to give them the desired preferred direction. 
    """
    #Damped oscillator parameters
    freqs = (2*np.pi)/tcs  #frequencies, Rad/s
    damp = -.75*freqs #corresponding damping values, 2 is critical damping
    V1_tau = 0.02
    V1sub=[]
    V1sub_probe=[]

    for j in range(n_dirs):
        cur_pop = []
        cur_probe = []
        for i in range(n_speeds):
            cur_pop.append(nengo.Ensemble(nengo.LIF(500), dimensions=2, label="V1_subpopulation_" + str(i) + "_" + str(j)))
            A = np.array([[damp[i], freqs[i]], [-freqs[i], damp[i]]])
#             A = np.array([[damp[0], freqs[i]], [-freqs[i], damp[0]]]) ###########

            #make it an oscillator
            c = nengo.Connection(cur_pop[i], cur_pop[i], transform=(V1_tau*A+np.eye(2)), filter=V1_tau) 
            # cur_probe.append(nengo.Probe(cur_pop[i],'decoded_output', filter=MT_tau)) # set filter=0 to debug nonlinearity 
        V1sub_probe.append(cur_probe)
        V1sub.append(cur_pop)
    return V1sub, V1sub_probe

def MT_subpopulation():
    """
    Simple group of populations that are meant to prefer different speeds and 
    directions due to converging input from V1 populations with the same 
    preferences and tiled spatial receptive fields.
    """
    MTsub=[]
    MTsub_probe=[]
    
    for j in range(n_dirs):
        cur_pop = []
        cur_probe = []
        for i in range(n_speeds):
            cur_pop.append(nengo.Ensemble(nengo.LIF(200), dimensions=1, label='MT'+str(j)+str(i)))
            cur_probe.append(nengo.Probe(cur_pop[i],'decoded_output', filter=MT_tau))
        MTsub_probe.append(cur_probe)
        MTsub.append(cur_pop)
    return MTsub, MTsub_probe
        

In [ ]:
MT_nodes, MT_probes = MT_subpopulation()

def V1_MT_fun(x): 
    """
    Nonlinear function of oscillator state to be decoded for MT input. 
    """
    direction = np.sqrt(.5) * np.array([-1, 1])    
    r = np.dot(x, direction)
    return 4*np.maximum(0, r-.4)

def passthrough_fun(t, x): 
    return x

V1_nodes = []
V1_probes = []
V1_nonlin_probes = []
for i in range(n): 
    print(i)
    nodes, probes = V1_subpopulation()
    V1_nodes.append(nodes)
    V1_probes.append(probes)
    
    # populations that code different speeds differ only in oscillator time constant
    # populations that code different directions differ in using left or right gabor
    nonlin_probes = [[], []]
    for j in range(len(nodes[0])): 
        transform = [[2,0,-.0],[0,-2,0]] # <-- for bumping the state with input at different phases
        nengo.Connection(input_nodes[i], nodes[0][j], transform=transform) 
        nengo.Connection(nodes[0][j], MT_nodes[0][j], transform=1, function=V1_MT_fun)
        dummy = nengo.Node(output=passthrough_fun, size_in=1, size_out=1, label="Dummy")
        nengo.Connection(nodes[0][j], dummy, transform=1, function=V1_MT_fun, filter=0)
        #nonlin_probes[0].append(nengo.Probe(dummy, 'output'))

    for j in range(len(nodes[1])): 
        transform = [[-.0,0,2],[0,-2,0]]
        nengo.Connection(input_nodes[i], nodes[1][j], transform=transform)
        nengo.Connection(nodes[1][j], MT_nodes[1][j], transform=1, function=V1_MT_fun)
        dummy = nengo.Node(output=passthrough_fun, size_in=1, size_out=1, label="Dummy")
        nengo.Connection(nodes[1][j], dummy, transform=1, function=V1_MT_fun, filter=0)
        #nonlin_probes[1].append(nengo.Probe(dummy, 'output'))

    V1_nonlin_probes.append(nonlin_probes)        

0


In [ ]:
if hardware:
    sim = nengo_rt_backend.Simulator(model, targetFile="/home/mtrberzi/nengo-rt-backend/target_1d2d.xml")
else:
    sim = nengo.Simulator(model)
print("Running simulation")
sim.run(1)

In [ ]:
plt.figure(figsize=(4*n_speeds, 4*n_dirs))
means = np.zeros((n_dirs, n_speeds))
for j in range(n_dirs):
    for i in range(n_speeds):
        ax = plt.subplot(n_dirs,n_speeds,(j*n_speeds)+i+1)
        plt.plot(sim.trange(), sim.data(MT_probes[j][i]))
        ax.set_ylim([-2,1.5])
        means[j,i] = np.average(sim.data(MT_probes[j][i]))
        
print(means)

In [ ]:
print(means)

In [ ]:
# debugging code ... 

pop_ind = 0

plt.figure(figsize=(4*n_speeds, 4*n_dirs))
for j in range(n_dirs):
    for i in range(n_speeds):
        ax = plt.subplot(n_dirs,n_speeds,(j*n_speeds)+i+1)
        plt.plot(sim.trange(), sim.data(V1_probes[pop_ind][j][i]))
        ax.set_ylim([-1,1])


plt.figure(figsize=(4*n_speeds, 4))
for j in range(n_dirs):
    colours = ('b', 'r')
    for i in range(n_speeds):
#         ax = plt.subplot(n_dirs,n_speeds,(j*n_speeds)+i+1)
        ax = plt.subplot(1,n_speeds,i+1)
        plt.plot(sim.data(V1_probes[pop_ind][j][i])[:,0], sim.data(V1_probes[pop_ind][j][i])[:,1], colours[j])
        ax.set_xlim([-1,1])
        ax.set_ylim([-1,1])


In [ ]:
# more debugging code ... 

speed_ind = 0

plt.figure(figsize=(6,6))
plt.plot(sim.trange(), np.squeeze(sim.data(V1_nonlin_probes[pop_ind][0][speed_ind])), 'b')
plt.plot(sim.trange(), np.squeeze(sim.data(V1_nonlin_probes[pop_ind][1][speed_ind])), 'r')
                                                            
foo = sim.data(V1_probes[pop_ind][1][speed_ind])
bar = np.zeros(foo.shape[0])
for i in range(foo.shape[0]): 
    bar[i] = V1_MT_fun(foo[i,:])
  
plt.plot(sim.trange(), bar, 'r.')


In [ ]:
foo = np.squeeze(inputs[6,:,0:1000])
print(foo.shape)

plt.plot(foo.T)